In [1]:
from pyspark import SparkContext
import time
import json
import sys

# Task 1

## Case 1

In [2]:
sc = SparkContext.getOrCreate()

23/02/18 15:34:31 WARN Utils: Your hostname, Daniels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.26.160.85 instead (on interface en0)
23/02/18 15:34:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/18 15:34:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/18 15:34:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/18 15:34:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
filepath = '../resource/asnlib/publicdata/small1.csv'
# filepath = '../resource/asnlib/publicdata/ta_feng_all_months_merged.csv'

In [4]:
# Read File, skipping header
review = sc.textFile(filepath).zipWithIndex().filter(lambda x: x[1] > 0).map(lambda line: line[0].split(","))

In [5]:
review = review.map(lambda x: (x[0], x[1]))

In [6]:
user_basket = review.groupByKey().mapValues(set).map(lambda x: (x[0], list(x[1])))

In [7]:
user_basket_memory = user_basket.collect()

**A-priori algorithm**

In [8]:
user_basket_memory[:5]


[('1', ['102', '100', '101', '98']),
 ('4', ['102', '97', '101', '99', '103']),
 ('8', ['102', '97', '104', '99', '103', '98']),
 ('9', ['99', '97', '98']),
 ('10', ['97', '98'])]

In [9]:
from collections import defaultdict

In [10]:
def hash_items(basket_rdd: list) -> dict:
    '''Translates item names into integers from 1 to n

    Args:
        basket_rdd (list): [(key, value), ...]

    Returns:
        (dict): {item: hash, ...}
    '''
    subset_items = set()
    for key, list_items in basket_rdd:
        subset_items = subset_items | set(list_items)
    subset_items = list(subset_items)
    n = range(1, len(subset_items) + 1) # n is the number of distinct items
    map_to_int = { item: n_item for item, n_item in zip(subset_items, n)}
    return map_to_int

In [11]:
# user_basket_memory

In [12]:
map_item_hash = hash_items(user_basket_memory)

In [13]:
user_basket_memory_hashed = []
for key, list_items in user_basket_memory:
    user_basket_memory_hashedlist = [map_item_hash[item] for item in list_items]
    user_basket_memory_key = key
    user_basket_memory_hashed.append((user_basket_memory_key, user_basket_memory_hashedlist))

In [14]:
def triangular_matrix_method():
    pass

In [15]:
import itertools

In [16]:
map_item_hash.values()

dict_values([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

In [17]:
# list(itertools.combinations(map_item_hash.values(), 2))

In [18]:
new_candidate = (1,2,3)

In [19]:
c = set(itertools.combinations(new_candidate, 2))

In [20]:
c = set([(1, 2)])

In [21]:
c

{(1, 2)}

In [127]:
c <= set([(2,3), (1,2), (1,3), (4,5)])

True

In [62]:
set((1,2) )

{1, 2}

In [63]:
L_1 = [(1,), (2,), (3,)]

In [71]:
{1} in set((1,2,3))

False

In [72]:
a = (1,)

In [74]:
a[0]

1

In [69]:
set((1,2,3))

{1, 2, 3}

In [139]:
subset = (1,3)
#b,m

In [82]:
L_1 = [(1,), (3,), (4,), (2,)]
# b,c,j,m

In [319]:
# { {b,m} {b,c}  {c,m}  {c,j} }

In [83]:
L_k_1 = [(1,2), (1,3), (3,2), (3,4)]

In [129]:
L_k_1

[(1, 2), (1, 3), (3, 2), (3, 4)]

In [132]:
set([(1,2)]) <= set(L_k_1)

True

In [134]:
set([(2,1)]).intersection(set(L_k_1))

set()

In [101]:
L_k_1

[(1, 2), (1, 3), (3, 2), (3, 4)]

In [87]:
L_1

[(1,), (3,), (4,), (2,)]

In [150]:
subset

(1, 3)

In [148]:
a = new_candidate(subset, L_k_1, L_1, 3)

(1, 3, 4)
{(1, 3), (3, 4), (1, 4)}
(1, 3, 2)
{(3, 2), (1, 2), (1, 3)}


In [320]:
subset = (1,)
L_1 = [(1,), (3,), (4,), (2,)]
L_k_1 = [(1,), (3,), (4,), (2,)]
# b,c,j,m
#b,m

In [321]:
a = new_candidate(subset, L_k_1, L_1, 2)

TypeError: new_candidate() missing 1 required positional argument: 'C_k'

In [29]:
{3,1} in [{1,3}, {2,3}]

True

In [31]:
def new_candidate(subset: tuple, L_k_1: list, L_1: list, k: int, C_k: list) -> list:
    list_of_candidates = []
    for frequent_item in L_1: # Iterate over each item of L_1 that does not intersect
        if frequent_item[0] in subset: # Do nothing if L_1 item is already in subset
            continue
        # For each basket, we need only look at those items that are in L_1
        new_candidate = set(sorted(subset + frequent_item)) 
        
        # print(new_candidate)
        if new_candidate not in list_of_candidates:
            # Examine each pair and determine whether or not that pair is in L_k - 1
            subset_combination = set(itertools.combinations(new_candidate, k-1))
            # print(subset_combination)
            # print(subset_combination)
            if subset_combination <= set(L_k_1):
                list_of_candidates.append(tuple(new_candidate)) if new_candidate not in C_k else list_of_candidates # Remove duplicates
    return list_of_candidates

In [32]:
def generate_candidates(L_k_1: list, L_1: list, k: int):
    # Generatin C_k from L_k_1 and L_1
    C_k = []
    for subset in L_k_1:
        if subset == (1,3):
            new_subsets_candidates = new_candidate(subset, L_k_1, L_1, k, C_k)
            C_k.extend(new_subsets_candidates) #if new_subsets_candidates not in C_k else C_k
            # break

    return C_k

In [63]:
def count_itemsets_of_size_k(basket_rdd: list, candidates_list: list, threshold: int) -> list:
    '''
    Count k-size candidate itemsets and returns those that are frequent

    Args:
        basket_rdd (int)
        candidates_list (list)
        threshold (int)
    Returns:
        
    '''
    candidates_count = defaultdict(int)
    for _, item_list in basket_rdd:
        for candidate in candidates_list:
            if set(candidate) <= set(item_list):
                candidates_count[candidate] += 1 # increase 1 count

    return [k for k,v in candidates_count.items() if v >= threshold]
            

In [28]:
k = 1
threshold = 3
# K = 1
C_k = [(item,) for item in map_item_hash.values()] # Singletons Candidates
L_k_1 = map_item_hash.values() # all singletons

for k in range(1, 20):
    pass
    C_k = generate_candidates(L_k_1, L_1, k)

    

NameError: name 'L_1' is not defined

In [363]:
L_k_1

dict_values([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

In [364]:
set(1]) <= set((6,2,1))

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (2900746241.py, line 1)

In [230]:
{k:v for k,v in a.items() if v > 5}

{(3, 7): 10}

In [195]:
a = defaultdict(int)

In [201]:
a[[1,2,3]]

TypeError: unhashable type: 'list'

In [196]:
a[1]

0

In [171]:
set({'1'}) <= set(user_basket_memory[0][1])

False

In [86]:
# review = review.flatMap(lambda l: [(l[0], value) for value in l[1:]])

In [139]:
subset_items + {1}

TypeError: unsupported operand type(s) for +: 'set' and 'set'